In [ ]:
!pip install transformers accelerate datasets evaluate huggingface-hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "316usman/Llama-2-7b-chat-tapal-demo"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)
# length = len(tokenizer.tokenize(prompt))
# sequences = pipeline(
#     f'<s>[INST] {prompt} [/INST]',
#     do_sample=True,
#     top_k=10,
#     num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
#     max_length=length,
#     temperature = 0.01,
#     no_repeat_ngram_size = 10,
#     early_stopping = True
# )
# for seq in sequences:
#     print(f"Result: {seq['generated_text']}")


tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
def model_inference(prompt):
  prompt = prompt.replace('<s> ','')
  length = len(tokenizer.tokenize(prompt)) * 5
  sequences = pipeline(
      f'<s>[INST]{prompt}[/INST]',
      temperature = 0.1,
      do_sample=True,
      top_k=5,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_length= length,
      no_repeat_ngram_size = 10
  )
  for seq in sequences:
      print(seq['generated_text'])
      return(seq['generated_text'])

In [ ]:
from datasets import load_dataset
import torch
import pandas as pd
import gc
dataset = load_dataset("316usman/tapal_validation_dataset",split='train')

In [ ]:
generated_text = []
for prompt in dataset['half_text']:
  generated_text.append(model_inference(prompt))
  torch.cuda.empty_cache()
  gc.collect()
  gc.collect()


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
len(generated_text)

98

In [ ]:
dataset = dataset.add_column("generated_text_1", generated_text)

In [ ]:
for i in range(len(generated_text)):
  generated_text[i] = generated_text[i].replace('<s>','').replace('[INST]','').replace('[/INST]','')

In [ ]:
dataset.push_to_hub("316usman/tapal_validation_dataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/362 [00:00<?, ?B/s]